In [88]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
from torchvision import datasets, transforms;
import matplotlib.pyplot as plt
import numpy as np

# train set과 test set 불러오기
train_data = datasets.MNIST('./datasets', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('./datasets', train=False, download=True, transform=transforms.ToTensor())
# 한번에 batch_size  만큼의 데이터만 불러오게 하고, 순서를 섞어서 불러오게 해줌. 
batch_size = 12
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# Set the dtypes of parameters
weight_type = torch.float64
torch.set_default_dtype(weight_type)


# Define MLP neural network model
class MLP(nn.Module):
    def __init__(self, num_of_hidden_layers, num_of_neurons):
        super().__init__()
        self.in_dim = 28 * 28  # input dimension
        self.out_dim = 10  # output dimension (class 0~9)
        # 층 정의
        layers = [nn.Linear(self.in_dim, num_of_neurons), nn.ReLU()]
        for _ in range(num_of_hidden_layers):
            layers.append(nn.Linear(num_of_neurons, num_of_neurons))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(num_of_neurons, self.out_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x.view(-1, self.in_dim))

    def accuracy(self):
        n_predict = 0
        n_correct = 0
        wrong = []
        expected = []
        results = []
        for data in test_loader:
            inputs, labels = data
            outputs = self(inputs)
            _, predicted = torch.max(outputs, 1)
            n_predict += len(predicted)
            for label, pred, input in zip(labels, predicted, inputs):
                if label != pred:
                    wrong.append(input)
                    expected.append(pred)
                    results.append(label)
                else:
                    n_correct += 1
        return n_correct / n_predict

    def size(self):
        param_size = 0
        buffer_size = 0
        for param in self.parameters():
            param_size += param.nelement() * param.element_size()
        for buffer in self.buffers():
            buffer_size += buffer.nelement() * buffer.element_size()
        size_all_kb = (param_size + buffer_size) / 1024 ** 2
        return size_all_kb

    def _train(self):
        criterion = nn.CrossEntropyLoss()  # Loss function
        optimizer = optim.SGD(self.parameters(), lr=0.01)  # optimizer  
        for epoch in range(5):
            # batch iteration, batch_size가 12이고, MNIST 데이터는 6만개이므로 5000번의 iteration
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                self.zero_grad()  # 이전 batch iteration 에서 기울기 값이 누적 되지 않기 위해 기울기를 초기화
                outputs = self(inputs)
                loss = criterion(outputs, label)  # loss 계산
                loss.backward()  # 기울기 계산
                optimizer.step()  # 파라미터 업데이트
        print("Accuracy: {}".format((self.accuracy())))
        print('Size: {:.3f} MB'.format(self.size()))


for num_of_hidden_layers in [1, 3, 5, 7]:
    for num_of_neurons in [64, 32, 16, 8, 4]:
        print("model with {} layers and {} neurons".format(num_of_hidden_layers, num_of_neurons))
        model = MLP(num_of_hidden_layers, num_of_neurons)
        model._train()
        print("--------------------------------------")

model with 1 layers and 64 neurons
Accuracy: 0.9555
Size: 0.420 MB
--------------------------------------
model with 1 layers and 32 neurons



KeyboardInterrupt



In [91]:
import os

# Set the dtypes of parameters
weight_type = torch.float32
torch.set_default_dtype(weight_type)

model_fp32 = MLP(1, 64)
model_fp32._train()

model_int8 = torch.ao.quantization.quantize_dynamic(
    model_fp32,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)

n_predict = 0
n_correct = 0
wrong = []
expected = []
results = []
for data in test_loader:
    inputs, labels = data
    outputs = model_int8(inputs)
    _, predicted = torch.max(outputs, 1)
    n_predict += len(predicted)
    for label, pred, input in zip(labels, predicted, inputs):
        if label != pred:
            wrong.append(input)
            expected.append(pred)
            results.append(label)
        else:
            n_correct += 1



def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")
    print("model: ", label, ' \t', 'Size (KB):', size / 1e3)
    os.remove('temp.p')
    return size

print("Accuracy after quantiztion: {}".format((n_correct / n_predict)))
print_size_of_model(model_int8)

Accuracy: 0.9595
Size: 0.210 MB
Accuracy after quantiztion: 0.9595
model:    	 Size (KB): 59.714


59714